## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Multiple projects found. 

	 (1) ID2223_Ernest
	 (2) ID2223_Anton

Enter project to access: 1

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5476
Connected. Call `.close()` to terminate connection gracefully.


In [2]:
air_quality_fg = fs.get_or_create_feature_group(
    name = 'poland_air_quality_fg',
    version = 1
)
weather_fg = fs.get_or_create_feature_group(
    name = 'poland_weather_fg',
    version = 1
)
meta_fg = fs.get_or_create_feature_group(
    name = 'poland_meta_fg',
    version = 1
)

In [3]:
query = air_quality_fg.select(['date', 'pm10', 'city'])\
                              .join(weather_fg.select_all())\
                              .join(meta_fg.select(['area', 'density', 'no_people']))

In [4]:
df = query.read()

2023-01-13 15:26:44,423 INFO: USE `id2223_ernest_featurestore`
2023-01-13 15:26:44,879 INFO: SELECT `fg2`.`date` `date`, `fg2`.`pm10` `pm10`, `fg2`.`city` `city`, `fg0`.`tempmax` `tempmax`, `fg0`.`tempmin` `tempmin`, `fg0`.`temp` `temp`, `fg0`.`feelslikemax` `feelslikemax`, `fg0`.`feelslikemin` `feelslikemin`, `fg0`.`feelslike` `feelslike`, `fg0`.`dew` `dew`, `fg0`.`humidity` `humidity`, `fg0`.`precip` `precip`, `fg0`.`precipprob` `precipprob`, `fg0`.`precipcover` `precipcover`, `fg0`.`snow` `snow`, `fg0`.`snowdepth` `snowdepth`, `fg0`.`windgust` `windgust`, `fg0`.`windspeed` `windspeed`, `fg0`.`winddir` `winddir`, `fg0`.`cloudcover` `cloudcover`, `fg0`.`visibility` `visibility`, `fg0`.`solarradiation` `solarradiation`, `fg0`.`solarenergy` `solarenergy`, `fg0`.`uvindex` `uvindex`, `fg0`.`conditions` `conditions`, `fg1`.`area` `area`, `fg1`.`density` `density`, `fg1`.`no_people` `no_people`
FROM `id2223_ernest_featurestore`.`poland_air_quality_fg_1` `fg2`
INNER JOIN `id2223_ernest_featu

--- 

## <span style="color:#ff5f27;"> 🖍 Feature View Creation and Retrieving </span>

In [32]:
query_show = query.show(5)
col_names = query_show.columns

                                         
query_show

2023-01-12 21:13:35,094 INFO: USE `id2223_ernest_featurestore`
2023-01-12 21:13:35,783 INFO: SELECT `fg2`.`date` `date`, `fg2`.`pm10` `pm10`, `fg2`.`city` `city`, `fg0`.`tempmax` `tempmax`, `fg0`.`tempmin` `tempmin`, `fg0`.`temp` `temp`, `fg0`.`feelslikemax` `feelslikemax`, `fg0`.`feelslikemin` `feelslikemin`, `fg0`.`feelslike` `feelslike`, `fg0`.`dew` `dew`, `fg0`.`humidity` `humidity`, `fg0`.`precip` `precip`, `fg0`.`precipprob` `precipprob`, `fg0`.`precipcover` `precipcover`, `fg0`.`snow` `snow`, `fg0`.`snowdepth` `snowdepth`, `fg0`.`windgust` `windgust`, `fg0`.`windspeed` `windspeed`, `fg0`.`winddir` `winddir`, `fg0`.`cloudcover` `cloudcover`, `fg0`.`visibility` `visibility`, `fg0`.`solarradiation` `solarradiation`, `fg0`.`solarenergy` `solarenergy`, `fg0`.`uvindex` `uvindex`, `fg0`.`conditions` `conditions`, `fg1`.`area` `area`, `fg1`.`density` `density`, `fg1`.`no_people` `no_people`
FROM `id2223_ernest_featurestore`.`poland_air_quality_fg_1` `fg2`
INNER JOIN `id2223_ernest_featu

,date,pm10,city,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,winddir,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions,area,density,no_people
0,1652392800000,19.0,biała podlaska,20.6,7.3,14.9,20.6,7.3,14.9,6.2,...,253.9,68.7,27.0,293.1,25.4,8,Partially cloudy,49.4,1152.06,56942
1,1664488800000,35.0,biała podlaska,12.9,2.6,8.3,12.9,2.6,8.3,6.5,...,59.8,73.6,7.6,100.4,8.6,5,"Rain, Partially cloudy",49.4,1152.06,56942
2,1664920800000,18.0,biała podlaska,16.4,7.1,11.2,16.4,5.9,10.9,8.1,...,222.3,72.8,18.1,81.9,7.0,4,"Rain, Partially cloudy",49.4,1152.06,56942
3,1665525600000,42.0,biała podlaska,13.5,1.1,7.7,13.5,-0.5,7.3,4.5,...,264.8,57.8,11.3,129.3,11.1,5,Partially cloudy,49.4,1152.06,56942
4,1661032800000,10.0,biała podlaska,23.7,16.9,19.9,23.7,16.9,19.9,16.7,...,80.1,79.1,23.7,175.1,15.1,7,Partially cloudy,49.4,1152.06,56942


In [33]:
category_cols = ['city','date','conditions']

pred_col = "pm10"
mapping_transformers = {col_name:fs.get_transformation_function(name='standard_scaler') for col_name in col_names if col_name not in category_cols and col_name != pred_col}
category_cols = {col_name:fs.get_transformation_function(name='label_encoder') for col_name in category_cols if col_name not in ['date','aqi']}

mapping_transformers.update(category_cols)

In [34]:
feature_view = fs.create_feature_view(
    name = 'poland_air_quality_fv',
    version = 1,
    transformation_functions = mapping_transformers,
    query = query
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/5476/fs/5383/fv/poland_air_quality_fv/version/1


For now `Feature View` is saved in Hopsworks and you can retrieve it using `FeatureStore.get_feature_view()`.

In [23]:
feature_view = fs.get_feature_view(
    name = 'poland_air_quality_fv',
    version = 1
)

In [37]:
# note: I've noticed that it fails randomly. Run the creation of the training dataset within the hopsworks UI
#feature_view.create_training_data()